In [127]:
import re
import math
import warnings
import tqdm
import json
import matplotlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy import optimize


warnings.filterwarnings("ignore")

In [128]:
data1 = pd.read_excel('治疗方案.xlsx') # 治疗方案
data2 = pd.read_excel('含分组信息数据_水肿.xlsx') # 水肿数据

In [129]:
data3 = pd.merge(data1, data2, how='outer', on='ID')
data3 = data3.drop(range(101, 131))

In [130]:
data3.columns

Index(['ID', '['止吐护胃', '营养神经']', '['降颅压治疗', '降压治疗']', '['止血治疗', '止吐护胃']',
       '['止血治疗', '降颅压治疗', '降压治疗']', '['止血治疗', '降压治疗', '营养神经']',
       '['止血治疗', '镇静、镇痛治疗', '止吐护胃']', '['降颅压治疗', '降压治疗', '止吐护胃']',
       '['镇静、镇痛治疗', '止吐护胃', '营养神经']', '['降压治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '止吐护胃', '营养神经']', '['降颅压治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降颅压治疗', '止吐护胃']', '['降颅压治疗', '降压治疗', '营养神经']',
       '['止血治疗', '降颅压治疗', '止吐护胃', '营养神经']', '['脑室引流', '降压治疗', '止吐护胃', '营养神经']',
       '['降颅压治疗', '降压治疗', '止吐护胃', '营养神经']',
       '['降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降颅压治疗', '降压治疗', '营养神经']', '['止血治疗', '降压治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']',
       '['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']',
       '['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']',
       '['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']',
       '['止血治疗', '降颅压治疗', '降

In [131]:
import json
with open('best_labels_130.json','r') as f:
    groups_label=json.load(f)
print(len(groups_label['best_labels_130']))

130


In [132]:
dataT=data3
dataT['分组信息']=groups_label['best_labels_130']
dataT.to_excel('含分组信息数据.xlsx',index=False)

In [133]:
# 创建一个新列"差"，初始化为0
dataT['差'] = 0
U=globals()
res={}
for i in list(set(dataT['分组信息'])):
    U[f'data_g{i}']=dataT[dataT['分组信息']==i]

# 遍历数据集的每一行
    for index, row in U[f'data_g{i}'].iterrows():
        row_sum = 0  # 用于累积每行的差值

        prev_value = None  # 用于存储前一个以"ED"开头的列的值

        # 遍历以"ED"开头的列
        for col in row.index:
            if col.startswith('ED'):
                value = row[col]
                if not pd.isna(value):  # 确保元素不是NaN
                    if prev_value is not None:
                        row_sum += (value - prev_value)  # 计算差值并累积
                    prev_value = value

        # 根据row_sum的值设置"差"列的值
        if row_sum > 0:
            U[f'data_g{i}'].at[index, '差'] = 1
        elif row_sum < 0:
            U[f'data_g{i}'].at[index, '差'] = -1
        res[f'组{i}']=float(U[f'data_g{i}']['差'].sum())

res_res = {}
for i in res:
    if res[i] < 0:
        res_res[i]=-1
    else:
        res_res[i]=1
# with open('分组趋势.json', 'w') as f:
#     json.dump(res_res,f)

In [134]:
# with open('分组趋势.json', 'r') as f:
#     print(json.load(f))

In [135]:
res_res

{'组0': 1, '组1': 1, '组2': -1, '组3': -1, '组4': -1}

In [136]:
#################### 合并最终表

In [137]:
data4 = pd.read_excel("治疗方案.xlsx")
data4 = data4.drop(range(101, 131))
mapping_dict = {}
p = 1
for col in data4.columns:
    # 如果列名包含[符号，则进行转换
    if '[' in col and ']' in col:
        mapping_dict[f'治疗组合{p}'] = col  # 添加映射关系到字典
        # 更新DataFrame的列名
        data4.rename(columns={col: f'治疗组合{p}'}, inplace=True)
        p += 1
with open('mapping.json', 'w') as json_file:
    json.dump(mapping_dict, json_file, ensure_ascii=False)
# data4.columns
data4.head()

,ID,治疗组合1,治疗组合2,治疗组合3,治疗组合4,治疗组合5,治疗组合6,治疗组合7,治疗组合8,治疗组合9,...,治疗组合19,治疗组合20,治疗组合21,治疗组合22,治疗组合23,治疗组合24,治疗组合25,治疗组合26,治疗组合27,治疗组合28
0,sub001,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
1,sub002,1,1,1,1,1,0,1,0,1,...,1,1,0,0,0,0,0,0,0,0
2,sub003,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,0,0,1,0,0
3,sub004,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,sub005,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [138]:
target = []
for count, key in enumerate(groups_label['best_labels_130']):
    value = res_res['组'+str(key)]
    target.append(value)

In [139]:
data4['组趋势'] = 0
data4['组趋势'] = np.array(target)
data4 = data4.drop(['ID'], axis=1)
data4.head()

,治疗组合1,治疗组合2,治疗组合3,治疗组合4,治疗组合5,治疗组合6,治疗组合7,治疗组合8,治疗组合9,治疗组合10,...,治疗组合20,治疗组合21,治疗组合22,治疗组合23,治疗组合24,治疗组合25,治疗组合26,治疗组合27,治疗组合28,组趋势
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
1,1,1,1,1,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,-1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1
4,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1


In [140]:
import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data4.drop('组趋势', axis=1)
Y = data4['组趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗水肿_regression_summary_组合_组趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                    组趋势   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 3.712e+28
Date:                Sun, 24 Sep 2023   Prob (F-statistic):               0.00
Time:                        16:47:05   Log-Likelihood:                 4002.4
No. Observations:                 130   AIC:                            -7953.
Df Residuals:                     104   BIC:                            -7878.
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.9077   6.81e-15  -1.33e+14      0.0

In [141]:
# data1 = pd.read_excel('2c治疗方案.xlsx') # 治疗方案
# 做单一治疗方法的
data5 = pd.concat([data1, data4[['组趋势']]], axis=1).drop(['ID'], axis=1)[:100]
data5

,"['止吐护胃', '营养神经']","['降颅压治疗', '降压治疗']","['止血治疗', '止吐护胃']","['止血治疗', '降颅压治疗', '降压治疗']","['止血治疗', '降压治疗', '营养神经']","['止血治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '止吐护胃']","['镇静、镇痛治疗', '止吐护胃', '营养神经']","['降压治疗', '止吐护胃', '营养神经']","['止血治疗', '止吐护胃', '营养神经']",...,"['止血治疗', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃']","['降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '止吐护胃', '营养神经']","['止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']","['脑室引流', '止血治疗', '降颅压治疗', '降压治疗', '镇静、镇痛治疗', '止吐护胃', '营养神经']",组趋势
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1.0
1,1,1,1,1,1,0,1,0,1,1,...,1,0,0,0,0,0,0,0,0,-1.0
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1.0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,-1.0
4,1,0,1,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,-1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,1,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,-1.0
96,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1.0
97,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,1,0,0,1.0
98,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1.0


In [142]:
import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('组趋势', axis=1)
Y = data5['组趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗水肿_regression_summary_单一_组趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                    组趋势   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 1.165e+29
Date:                Sun, 24 Sep 2023   Prob (F-statistic):               0.00
Time:                        16:47:06   Log-Likelihood:                 3132.0
No. Observations:                 100   AIC:                            -6228.
Df Residuals:                      82   BIC:                            -6181.
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

In [143]:
singel_target = []
for index, row in pd.read_excel('含分组信息数据.xlsx').iterrows():
    t = 0
    row_sum = 0  # 用于累积每行的差值
    prev_value = None  # 用于存储前一个以"ED"开头的列的值
    # 遍历以"ED"开头的列
    for col in row.index:
        if col.startswith('ED'):
            value = row[col]
            if not pd.isna(value):  # 确保元素不是NaN
                if prev_value is not None:
                    row_sum += (value - prev_value)  # 计算差值并累积
                prev_value = value
    if row_sum > 0:
        t = 1
    elif row_sum < 0:
        t = -1
    singel_target.append(t)

In [144]:

# 做单一趋势的
data5 = pd.concat([data1, pd.DataFrame(np.array(singel_target), columns = ['个人趋势'])], axis=1).drop(['ID'], axis=1)[:100]


In [145]:
# 做单独治疗无组合+个人趋势的

import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('个人趋势', axis=1)
Y = data5['个人趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗水肿_regression_summary_单一_个人趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                   个人趋势   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.9185
Date:                Sun, 24 Sep 2023   Prob (F-statistic):              0.555
Time:                        16:47:06   Log-Likelihood:                -133.18
No. Observations:                 100   AIC:                             302.4
Df Residuals:                      82   BIC:                             349.2
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                                                                   coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------

In [146]:
data5 = pd.concat([data4.drop(['组趋势'],axis=1),pd.DataFrame(np.array(singel_target), columns = ['个人趋势'])], axis = 1)[:100]





In [147]:
# 做组合+个人趋势的

import statsmodels.api as sm


# 选择自变量（X）和因变量（Y）
X = data5.drop('个人趋势', axis=1)
Y = data5['个人趋势']

# 2. 添加截距（常数项）
X = sm.add_constant(X)

# 3. 拟合多元线性回归模型
model = sm.OLS(Y, X).fit()

# 4. 获取回归结果摘要
summary = model.summary()

# 5. 打印结果摘要
print(summary)
# 指定保存文件的路径和名称
file_path = "q24治疗水肿_regression_summary_组合_个人趋势.txt"
# 打开文件以写入模式
with open(file_path, "w") as file:
    # 将回归结果摘要写入文件
    file.write(str(summary))
# 提示保存成功
print(f"回归结果摘要已保存到 {file_path}")

                            OLS Regression Results                            
Dep. Variable:                   个人趋势   R-squared:                       0.160
Model:                            OLS   Adj. R-squared:                 -0.014
Method:                 Least Squares   F-statistic:                    0.9185
Date:                Sun, 24 Sep 2023   Prob (F-statistic):              0.555
Time:                        16:47:07   Log-Likelihood:                -133.18
No. Observations:                 100   AIC:                             302.4
Df Residuals:                      82   BIC:                             349.2
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.2331      0.537     -0.434      0.6